# Developing Word Embeddings

Rather than use pre-trained embeddings (as we did in the baseline_deep_dive notebook), we can train word embeddings using our own dataset. In this notebook, we demonstrate the training process for producing word embeddings using the word2vec, GloVe, and fastText models. We'll utilize the STS Benchmark dataset for this task. 

# Table of Contents
* [Data Loading and Preprocessing](#Load-and-Preprocess-Data)
* [Word2Vec](#Word2Vec)
* [fastText](#fastText)
* [GloVe](#GloVe)
* [Concluding Remarks](#Concluding-Remarks)

In [1]:
import gensim
import sys
import os

## Load and Preprocess Data

In [2]:
# Set the environment path
sys.path.append("../../") 
# Set the path for where your datasets are located
BASE_DATA_PATH = "../../data" 
# Location to save embeddings
SAVE_FILES_PATH = BASE_DATA_PATH + "/trained_word_embeddings/"

if not os.path.exists(SAVE_FILES_PATH):
    os.makedirs(SAVE_FILES_PATH)
    
from utils_nlp.dataset.preprocess import (
    to_lowercase,
    to_spacy_tokens,
    rm_spacy_stopwords,
)
from utils_nlp.dataset import stsbenchmark

In [3]:
# Produce a pandas dataframe for the training set
sts_train = stsbenchmark.load_pandas_df(BASE_DATA_PATH, file_split="train")

#### Training set preprocessing

In [4]:
# Convert all text to lowercase
df_low = to_lowercase(sts_train)  
# Tokenize text
sts_tokenize = to_spacy_tokens(df_low) 
# Tokenize with removal of stopwords
sts_train_stop = rm_spacy_stopwords(sts_tokenize) 

In [7]:
# Append together the two sentence columns to get a list of all tokenized sentences
all_sentences =  sts_train_stop[["sentence1_tokens_rm_stopwords", "sentence2_tokens_rm_stopwords"]]
sentences = all_sentences.values.flatten().tolist()

In [8]:
sentences[:10]

[['plane', 'taking', '.'],
 ['air', 'plane', 'taking', '.'],
 ['man', 'playing', 'large', 'flute', '.'],
 ['man', 'playing', 'flute', '.'],
 ['man', 'spreading', 'shreded', 'cheese', 'pizza', '.'],
 ['man', 'spreading', 'shredded', 'cheese', 'uncooked', 'pizza', '.'],
 ['men', 'playing', 'chess', '.'],
 ['men', 'playing', 'chess', '.'],
 ['man', 'playing', 'cello', '.'],
 ['man', 'seated', 'playing', 'cello', '.']]

## Word2Vec

Word2vec is a predictive model for learning word embeddings from text. Word embeddings are learned such that words that share common contexts in the corpus will be close together in the vector space. There are two different model architectures that can be used to produce word2vec embeddings: continuous bag-of-words (CBOW) or continuous skip-gram. The former uses a window of surrounding words (the "context") to predict the current word and the latter uses the current word to predict the surrounding context words. See this [tutorial](https://www.guru99.com/word-embedding-word2vec.html#3) on word2vec for more detailed background on the model.

The gensim Word2Vec model has many different parameters (see [here](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec)) but the ones that are useful to know about are:  
- size: length of the word embedding/vector (defaults to 100)
- window: maximum distance between the word being predicted and the current word (defaults to 5)
- min_count: ignores all words that have a frequency lower than this value (defaults to 5)
- workers: number of worker threads used to train the model (defaults to 3)
- sg: training algorithm; 1 for skip-gram and 0 for CBOW (defaults to 0)

In [9]:
from gensim.models import Word2Vec

word2vec_model = Word2Vec(sentences, size=100, window=5, min_count=5, workers=3, sg=0)

Now that the model is trained we can:

1. Query for the word embeddings of a given word. 
2. Inspect the model vocabulary
3. Save the word embeddings

In [10]:
# 1. Let's see the word embedding for "apple" by accessing the "wv" attribute and passing in "apple" as the key.
print("Embedding for apple:", word2vec_model.wv["apple"])

# 2. Inspect the model vocabulary by accessing keys of the "wv.vocab" attribute. We'll print the first 20 words
print("\nFirst 30 vocabulary words:", list(word2vec_model.wv.vocab)[:20])

# 3. Save the word embeddings. We can save as binary format (to save space) or ASCII format
word2vec_model.wv.save_word2vec_format(SAVE_FILES_PATH+"word2vec_model", binary=True)  # binary format
word2vec_model.wv.save_word2vec_format(SAVE_FILES_PATH+"word2vec_model", binary=False)  # ASCII format

Embedding for apple: [ 0.02385123  0.07058834 -0.15363528 -0.09833426 -0.03912428 -0.09718188
  0.14712512  0.12437295  0.2426645   0.1706049   0.02995755 -0.15563945
 -0.22095586 -0.16030319  0.16076583  0.06203773  0.07827957  0.16496783
 -0.09863456  0.09413838 -0.02348543 -0.00786579 -0.19533783 -0.08082277
  0.13688101  0.11107839 -0.08636363 -0.13031363 -0.10180362 -0.10177843
  0.08569022 -0.00783137  0.18135263  0.20505813 -0.0255485  -0.09544634
 -0.11168214  0.11173216  0.04231942  0.04790898  0.20185575 -0.206185
 -0.00579798  0.0129708  -0.05664448  0.30893633  0.01850598  0.05381552
  0.07570266  0.08633997  0.08449002  0.19658448  0.13327192  0.03369491
  0.01117267 -0.05898936 -0.04606275 -0.22269456  0.09217231  0.01435243
 -0.13452472 -0.07815848 -0.15210707 -0.12388272  0.03126181  0.00068742
  0.01804273  0.00766948 -0.03006491 -0.0053426  -0.09792323  0.04392556
  0.06998863  0.09082907  0.0663078  -0.11197442  0.05921098 -0.0366937
 -0.0104116  -0.01657981  0.25991

## fastText

fastText is an unsupervised algorithm created by Facebook Research for efficiently learning word embeddings. fastText is significantly different than word2vec or GloVe in that these two algorithms treat each word as the smallest possible unit to find an embedding for. Conversely, fastText assumes that words are formed by an n-gram of characters (i.e. 2-grams of the word "language" would be {la, an, ng, gu, ua, ag, ge}). The embedding for a word is then composed of the sum of these character n-grams. This has advantages when finding word embeddings for rare words and words not present in the dictionary, as these words can still be broken down into character n-grams. Typically, for smaller datasets, fastText performs better than word2vec or GloVe. See this [tutorial](https://fasttext.cc/docs/en/unsupervised-tutorial.html) on fastText for more detail.

The gensim fastText model has many different parameters (see [here](https://radimrehurek.com/gensim/models/fasttext.html#gensim.models.fasttext.FastText)) but the ones that are useful to know about are:  
- size: length of the word embedding/vector (defaults to 100)
- window: maximum distance between the word being predicted and the current word (defaults to 5)
- min_count: ignores all words that have a frequency lower than this value (defaults to 5)
- workers: number of worker threads used to train the model (defaults to 3)
- sg: training algorithm- 1 for skip-gram and 0 for CBOW (defaults to 0)
- iter: number of epochs (defaults to 5)


In [11]:
from gensim.models.fasttext import FastText

fastText_model = FastText(size=100, window=5, min_count=5, sentences=sentences, iter=5)

We can utilize the same attributes as we saw above for word2vec due to them both originating from the gensim package

In [12]:
# 1. Let's see the word embedding for "apple" by accessing the "wv" attribute and passing in "apple" as the key.
print("Embedding for apple:", fastText_model.wv["apple"])

# 2. Inspect the model vocabulary by accessing keys of the "wv.vocab" attribute. We'll print the first 20 words
print("\nFirst 30 vocabulary words:", list(fastText_model.wv.vocab)[:20])

# 3. Save the word embeddings. We can save as binary format (to save space) or ASCII format
fastText_model.wv.save_word2vec_format(SAVE_FILES_PATH+"fastText_model", binary=True)  # binary format
fastText_model.wv.save_word2vec_format(SAVE_FILES_PATH+"fastText_model", binary=False)  # ASCII format

Embedding for apple: [ 0.25968415  0.06153858  0.5517546  -0.10831144 -0.13662393  0.37149438
 -0.1854391  -0.10157733 -0.542075    0.3100342   0.15336306  0.0746365
  0.53819436  0.2898559   0.001645    0.38930708 -0.3888089  -0.32883227
  0.03838186  0.09121272  0.19442275  0.30711967  0.04910801 -0.26433834
  0.15336548 -0.13941264 -0.1683031  -0.02305422  0.40802833 -0.08694664
  0.0693102  -0.22281072  0.30042285  0.18769404  0.3555825  -0.2173373
 -0.01920168  0.83874726  0.08311833 -0.01971375 -0.23151635  0.04117979
 -0.30657613  0.09439644 -0.05001464  0.08719175 -0.14741863 -0.27816507
 -0.1760938   0.06337793 -0.2824583   0.16904551  0.13089176  0.27594844
 -0.04133933 -0.29206607  0.09222366  0.4444458   0.25669447 -0.02970394
  0.22482945  0.02552024  0.19697818  0.14577016  0.2870848  -0.5179852
  0.4016686   0.08828393 -0.08436926 -0.21839246  0.08788402 -0.02710382
  0.02686991  0.28628653  0.08487599 -0.15779638  0.11426484  0.0914265
 -0.07413616  0.0361906   0.509633

## GloVe

GloVe is an unsupervised algorithm for obtaining word embeddings created by the Stanford NLP group. Training occurs on word-word co-occurrence statistics with the objective of learning word embeddings such that the dot product of two words' embeddings is equal to the words' probability of co-occurrence. See this [tutorial](https://nlp.stanford.edu/projects/glove/) on GloVe for more detailed background on the model. 

Gensim doesn't have an implementation of the GloVe model, so we suggest getting the code directly from the Stanford NLP [repo](https://github.com/stanfordnlp/GloVe). Run the following commands to clone the repo and then make. Clone the repo in the same location as this notebook! Otherwise, the paths below will need to be modified.  

    git clone http://github.com/stanfordnlp/glove    
    cd glove && make  

### Train GloVe vectors

Training GloVe embeddings requires some data prep and then 4 steps (also documented in the original Stanford NLP repo [here](https://github.com/stanfordnlp/GloVe/tree/master/src)).

**Step 0: Prepare Data**
   
In order to train our GloVe vectors, we first need to save our corpus as a text file with all words separated by 1+ spaces or tabs. Each document/sentence is separated by a new line character.

In [13]:
# Save our corpus as tokens delimited by spaces with new line characters in between sentences
with open(BASE_DATA_PATH+'/clean/stsbenchmark/training-corpus-cleaned.txt', 'w', encoding='utf8') as file:
    for sent in sentences:
        file.write(" ".join(sent) + "\n")

**Step 1: Build Vocabulary**

Run the vocab_count executable. There are 3 optional parameters:
1. min-count: lower limit on how many times a word must appear in dataset. Otherwise the word is discarded from our vocabulary.
2. max-vocab: upper bound on the number of vocabulary words to keep
3. verbose: 0, 1, or 2 (default)

Then provide the path to the text file we created in Step 0 followed by a file path that we'll save the vocabulary to 

In [20]:
!"glove/build/vocab_count" -min-count 5 -verbose 2 <"../../data/clean/stsbenchmark/training-corpus-cleaned.txt"> "../../data/trained_word_embeddings/vocab.txt"

BUILDING VOCABULARY
Processed 0 tokens.Processed 84997 tokens.
Counted 11716 unique words.
Truncating vocabulary at min count 5.
Using vocabulary of size 2943.



**Step 2: Construct Word Co-occurrence Statistics**

Run the cooccur executable. There are many optional parameters, but we list the top ones here:
1. symmetric: 0 for only looking at left context, 1 (default) for looking at both left and right context
2. window-size: number of context words to use (default 15)
3. verbose: 0, 1, or 2 (default)
4. vocab-file: path/name of the vocabulary file created in Step 1
5. memory: soft limit for memory consumption, default 4
6. max-product: limit the size of dense co-occurrence array by specifying the max product (integer) of the frequency counts of the two co-occurring words

Then provide the path to the text file we created in Step 0 followed by a file path that we'll save the co-occurrences to

In [21]:
!"glove/build/cooccur" -memory 4 -vocab-file "../../data/trained_word_embeddings/vocab.txt" -verbose 2 -window-size 15 <"../../data/clean/stsbenchmark/training-corpus-cleaned.txt"> "../../data/trained_word_embeddings/cooccurrence.bin"

COUNTING COOCCURRENCES
window size: 15
context: symmetric
max product: 13752509
overflow length: 38028356
Reading vocab from file "../../data/trained_word_embeddings/vocab.txt"...loaded 2943 words.
Building lookup table...table contains 8661250 elements.
Processing token: 0Processed 84997 tokens.
Writing cooccurrences to disk......2 files in total.
Merging cooccurrence files: processed 0 lines.0 lines.100000 lines.Merging cooccurrence files: processed 187717 lines.



**Step 3: Shuffle the Co-occurrences**

Run the shuffle executable. The parameters are as follows:
1. verbose: 0, 1, or 2 (default)
2. memory: soft limit for memory consumption, default 4
3. array-size: limit to the length of the buffer which stores chunks of data to shuffle before writing to disk

Then provide the path to the co-occurrence file we created in Step 2 followed by a file path that we'll save the shuffled co-occurrences to

In [22]:
!"glove/build/shuffle" -memory 4 -verbose 2 <"../../data/trained_word_embeddings/cooccurrence.bin"> "../../data/trained_word_embeddings/cooccurrence.shuf.bin"

SHUFFLING COOCCURRENCES
array size: 255013683
Shuffling by chunks: processed 0 lines.processed 187717 lines.
Wrote 1 temporary file(s).
Merging temp files: processed 0 lines.187717 lines.Merging temp files: processed 187717 lines.



**Step 4: Train GloVe model**

Run the glove executable. There are many parameter options, but the top ones are listed below:
1. verbose: 0, 1, or 2 (default)
2. vector-size: dimension of word embeddings (50 is default)
3. threads: number threads, default 8
4. iter: number of iterations, default 25
5. eta: learning rate, default 0.05
6. binary: whether to save binary format (0: text = default, 1: binary, 2: both)
7. x-max: cutoff for weighting function, default is 100
8. vocab-file: file containing vocabulary as produced in Step 1
9. save-file: filename to save vectors to 
10. input-file: filename with co-occurrences as returned from Step 3

In [23]:
!"glove/build/glove" -save-file "../../data/trained_word_embeddings/GloVe_vectors" -threads 8 -input-file \
"../../data/trained_word_embeddings/cooccurrence.shuf.bin" -x-max 10 -iter 15 -vector-size 50 -binary 2 \
-vocab-file "../../data/trained_word_embeddings/vocab.txt" -verbose 2

TRAINING MODEL
Read 187717 lines.
Initializing parameters...done.
vector size: 50
vocab size: 2943
x_max: 10.000000
alpha: 0.750000
05/01/19 - 04:02.27PM, iter: 001, cost: 0.078336
05/01/19 - 04:02.27PM, iter: 002, cost: 0.072057
05/01/19 - 04:02.27PM, iter: 003, cost: 0.070047
05/01/19 - 04:02.27PM, iter: 004, cost: 0.067139
05/01/19 - 04:02.27PM, iter: 005, cost: 0.063639
05/01/19 - 04:02.27PM, iter: 006, cost: 0.060582
05/01/19 - 04:02.27PM, iter: 007, cost: 0.058132
05/01/19 - 04:02.27PM, iter: 008, cost: 0.056071
05/01/19 - 04:02.27PM, iter: 009, cost: 0.054029
05/01/19 - 04:02.27PM, iter: 010, cost: 0.051805
05/01/19 - 04:02.27PM, iter: 011, cost: 0.049627
05/01/19 - 04:02.27PM, iter: 012, cost: 0.047423
05/01/19 - 04:02.27PM, iter: 013, cost: 0.045255
05/01/19 - 04:02.27PM, iter: 014, cost: 0.043173
05/01/19 - 04:02.27PM, iter: 015, cost: 0.041176


### Inspect Word Vectors

Like we did above for the word2vec and fastText models, let's now inspect our word embeddings

In [24]:
#load in the saved word vectors
glove_wv = {}
with open("../../data/trained_word_embeddings/GloVe_vectors.txt", encoding='utf-8') as f:
    for line in f:
        split_line = line.split(" ")
        glove_wv[split_line[0]] = [float(i) for i in split_line[1:]]

In [25]:
# 1. Let's see the word embedding for "apple" by passing in "apple" as the key.
print("Embedding for apple:", glove_wv["apple"])

# 2. Inspect the model vocabulary by accessing keys of the "wv.vocab" attribute. We'll print the first 20 words
print("\nFirst 30 vocabulary words:", list(glove_wv.keys())[:20])

Embedding for apple: [0.110111, -0.05142, 0.060918, 0.107242, 0.053554, -0.121755, 0.027269, 0.021243, -0.062346, 0.058418, -0.039505, 0.152648, 0.027803, 0.077568, 0.043909, -0.070705, 0.049434, -0.076611, 0.046025, 0.000879, 0.124484, 0.144175, 0.064219, 0.124845, -0.228454, 0.01921, 0.018549, 0.132425, -0.125736, -0.06528, 0.067697, -0.062586, -0.08961, 0.063824, -0.007195, -0.017496, 0.020269, -0.045943, -0.020522, -0.054329, 0.039319, -0.028744, 0.01779, -0.119619, 0.129229, 0.096591, -0.069199, 0.050528, -0.184094, -0.000369]

First 30 vocabulary words: ['.', ',', 'man', '-', 'woman', "'", 'said', 'dog', '"', 'playing', ':', 'white', 'black', '$', 'killed', 'percent', 'new', 'syria', 'people', 'china']


# Concluding Remarks

In this notebook we have shown how to train word2vec, GloVe, and fastText word embeddings on the STS Benchmark dataset. FastText is typically regarded as the best baseline for word embeddings (see [blog](https://medium.com/huggingface/universal-word-sentence-embeddings-ce48ddc8fc3a)) and is a good place to start when generating word embeddings. Now that we generated word embeddings on our dataset, we could also repeat the baseline_deep_dive notebook using these embeddings (versus the pre-trained ones from the internet). 